In [9]:
import pandas as pd
from tqdm import tqdm

In [5]:
train = pd.read_csv("../data/santander/train.csv")
test = pd.read_csv("../data/santander/test.csv")

In [17]:
test.head()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190_u,var_191_u,var_192_u,var_193_u,var_194_u,var_195_u,var_196_u,var_197_u,var_198_u,var_199_u
0,test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,...,False,False,False,False,False,False,False,False,False,False
1,test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,...,False,False,False,False,False,False,False,False,False,False
2,test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,...,False,False,False,False,False,False,False,False,False,False
3,test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,...,False,False,False,False,False,False,True,True,False,False
4,test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,...,False,False,False,False,False,False,False,False,False,False


In [18]:
col_names = [f"var_{i}" for i in range(200)]
for col in tqdm(col_names):
    count = test[col].value_counts()
    uniques = count.index[count == 1]
    test[col + "_u"] = test[col].isin(uniques)
    
test["has_unique"] = test[[col + "_u" for col in col_names]].any(axis=1)

100%|█████████████████████████████████████████| 200/200 [00:04<00:00, 48.12it/s]


In [20]:
test["has_unique"].sum()

100000

In [21]:
real_test = test.loc[test["has_unique"], ["ID_code"] + col_names]
fake_test = test.loc[~test["has_unique"], ["ID_code"] + col_names]
train_and_test = pd.concat([train, real_test], axis=0)

In [23]:
for col in tqdm(col_names):
    count = train_and_test[col].value_counts().to_dict()
    train_and_test[col+"_unique"] = train_and_test[col].apply(
        lambda x: 1 if count[x] == 1 else 0).values
    fake_test[col+"_unique"] = 0

 48%|████████████████████▎                     | 97/200 [00:24<00:24,  4.29it/s]/var/folders/v3/xcw2clc96dg337k24fyqm2h00000gn/T/ipykernel_75081/1854464992.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  train_and_test[col+"_unique"] = train_and_test[col].apply(
 49%|████████████████████▌                     | 98/200 [00:25<00:26,  3.82it/s]/var/folders/v3/xcw2clc96dg337k24fyqm2h00000gn/T/ipykernel_75081/1854464992.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  fake_test[col+"_unique"] = 0
100%|███████████████████████

In [24]:
real_test = train_and_test[train_and_test["ID_code"].str.contains("test")].copy()
real_test.drop(["target"], axis=1, inplace=True)
train = train_and_test[train_and_test["ID_code"].str.contains("train")].copy()

In [25]:
test = pd.concat([real_test, fake_test], axis=0)

In [29]:
train.to_csv("../data/santander/new_train.csv", index = False)
test.to_csv("../data/santander/new_test.csv", index = False)

In [26]:
train

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190_unique,var_191_unique,var_192_unique,var_193_unique,var_194_unique,var_195_unique,var_196_unique,var_197_unique,var_198_unique,var_199_unique
0,train_0,0.0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,0,0,0,0,0,0,0,0,0,0
1,train_1,0.0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,0,0,0,0,0,0,0,0,0,0
2,train_2,0.0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,0,0,0,0,0,0,0,0,0,0
3,train_3,0.0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,0,0,0,0,0,0,0,0,0,0
4,train_4,0.0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,0,0,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,train_199995,0.0,11.4880,-0.4956,8.2622,3.5142,10.3404,11.6081,5.6709,15.1516,...,0,1,0,0,0,0,1,0,0,0
199996,train_199996,0.0,4.9149,-2.4484,16.7052,6.6345,8.3096,-10.5628,5.8802,21.5940,...,0,0,0,1,0,0,0,0,0,0
199997,train_199997,0.0,11.2232,-5.0518,10.5127,5.6456,9.3410,-5.4086,4.5555,21.5571,...,0,0,0,0,0,0,0,0,0,1
199998,train_199998,0.0,9.7148,-8.6098,13.6104,5.7930,12.5173,0.5339,6.0479,17.0152,...,0,0,0,0,0,0,0,0,0,1
